In [20]:
from konlpy.tag import Okt
import json
import os
import numpy as np
import pandas as pd
from pprint import pprint

In [21]:
def preprocessing(filename) :
    df = pd.read_csv(filename , encoding='CP949')
    for i in df['total_x'] :
        if',' in i :
            i = i.replace(',' , '')
    data = []
    for i, row in df.iterrows():
        data.append([row[0],row[1]])
    
    return data

train_data = preprocessing('train.csv')
test_data = preprocessing('test.csv')
# print(train_data)
# print(test_data)

In [22]:
okt = Okt()

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [23]:
if os.path.isfile('train_docs.json'):
    with open('train_docs.json' , encoding='UTF8') as f:
        train_docs = json.load(f)
    with open('test_docs.json' , encoding='UTF8') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[0]), row[1]) for row in train_data]
    test_docs = [(tokenize(row[0]), row[1]) for row in test_data]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [24]:
tokens = [t for d in train_docs for t in d[0]]

In [25]:
import nltk
text = nltk.Text(tokens, name='NMSC')
# print(len(text.tokens))
# print(len(set(text.tokens)))

In [26]:
selected_words = [f[0] for f in text.vocab().most_common(1000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [27]:
train_x = [term_frequency(d) for d, _ in train_docs]
train_y = [c for _, c in train_docs]

In [28]:
test_x = [term_frequency(d) for d, _ in test_docs]
test_y = [c for _, c in test_docs]

In [29]:
x_train = np.asarray(train_x).astype('float32')
y_train = np.asarray(train_y).astype('float32')

In [30]:
x_test = np.asarray(test_x).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [31]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [46]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(1000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=100)

Epoch 1/10
36/36 [==============================] - 0s 8ms/step - loss: 0.4423 - binary_accuracy: 0.8233
Epoch 2/10
36/36 [==============================] - 0s 1ms/step - loss: 0.1768 - binary_accuracy: 0.9217
Epoch 3/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0897 - binary_accuracy: 0.9696
Epoch 4/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0586 - binary_accuracy: 0.9818
Epoch 5/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0427 - binary_accuracy: 0.9864
Epoch 6/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0310 - binary_accuracy: 0.9912
Epoch 7/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0235 - binary_accuracy: 0.9940
Epoch 8/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0184 - binary_accuracy: 0.9952
Epoch 9/10
36/36 [==============================] - 0s 1ms/step - loss: 0.0144 - binary_accuracy: 0.9957
Epoch 10/10
36/36 [==============================] - 0s

In [47]:
results = model.evaluate(x_test, y_test)

55/55 [==============================] - 0s 691us/step - loss: 0.1093 - binary_accuracy: 0.9647


In [50]:
def predict(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    print("[{}]는 업무일 확률 {:.2f}%.\n".format(review, score * 100))
    if(score > 0.75):
        print("업무O.\n")
    else:
        print("업무X.\n")

In [52]:
predict('내일 오후 4시까지 햄버거 사오세요.')
predict('요새 재미있는게 없을까?')
predict('테스트 해볼까?')
predict('와이어프레임 구성 완료했습니다')
predict('오늘 3시까지 백엔드 일정추가 기능 완료해주세요!')
predict('오늘까지 기획안 올려주세요')
predict('피드백받은 부분 반영해서 ppt 수정하였습니다. ')
predict('스토리 만들고, 스토리포인트 지정하고, 스프린트 시작해 주셔야 합니다! ')
predict('스토리 만들고, 스토리포인트 지정하고, 스프린트 시작하겠습니다! ')
predict('배고파 쩝쩝')

[내일 오후 4시까지 햄버거 사오세요.]는 업무일 확률 99.99%.

업무O.

[요새 재미있는게 없을까?]는 업무일 확률 0.11%.

업무X.

[테스트 해볼까?]는 업무일 확률 70.22%.

업무X.

[와이어프레임 구성 완료했습니다]는 업무일 확률 99.36%.

업무O.

[오늘 3시까지 백엔드 일정추가 기능 완료해주세요!]는 업무일 확률 100.00%.

업무O.

[오늘까지 기획안 올려주세요]는 업무일 확률 99.99%.

업무O.

[피드백받은 부분 반영해서 ppt 수정하였습니다. ]는 업무일 확률 100.00%.

업무O.

[스토리 만들고, 스토리포인트 지정하고, 스프린트 시작해 주셔야 합니다! ]는 업무일 확률 99.93%.

업무O.

[스토리 만들고, 스토리포인트 지정하고, 스프린트 시작하겠습니다! ]는 업무일 확률 99.98%.

업무O.

[배고파 쩝쩝]는 업무일 확률 0.06%.

업무X.



In [71]:
from tensorflow import keras
model.save('ToDoList.h5')

In [72]:
new_model = keras.models.load_model('ToDoList.h5')

In [74]:
def predict2(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(new_model.predict(data))
    print("[{}]는 업무일 확률 {:.2f}%.\n".format(review, score * 100))
    if(score > 0.75):
        print("업무O.\n")
    else:
        print("업무X.\n")

In [78]:
predict2('내일 오후 4시까지 햄버거 사오세요.')
predict2('요새 재미있는게 없을까?')
predict2('테스트 해볼까?')
predict2('와이어프레임 구성 완료했습니다')
predict2('오늘 3시까지 백엔드 일정추가 기능 완료해주세요!')
predict2('오늘까지 기획안 올려주세요')
predict2('피드백받은 부분 반영해서 ppt 수정하였습니다. ')
predict2('스토리 만들고, 스토리포인트 지정하고, 스프린트 시작해 주셔야 합니다! ')
predict2('스토리 만들고, 스토리포인트 지정하고, 스프린트 시작하겠습니다! ')
predict2('배고파 쩝쩝')
predict2('식당좀 알아봐주세요')
predict2('오늘 회식있는데 괜찮은 식당 좀 알아봐주세요')
predict2('괜찮은 식당 좀 알아봐주세요')

[내일 오후 4시까지 햄버거 사오세요.]는 업무일 확률 99.99%.

업무O.

[요새 재미있는게 없을까?]는 업무일 확률 0.11%.

업무X.

[테스트 해볼까?]는 업무일 확률 70.22%.

업무X.

[와이어프레임 구성 완료했습니다]는 업무일 확률 99.36%.

업무O.

[오늘 3시까지 백엔드 일정추가 기능 완료해주세요!]는 업무일 확률 100.00%.

업무O.

[오늘까지 기획안 올려주세요]는 업무일 확률 99.99%.

업무O.

[피드백받은 부분 반영해서 ppt 수정하였습니다. ]는 업무일 확률 100.00%.

업무O.

[스토리 만들고, 스토리포인트 지정하고, 스프린트 시작해 주셔야 합니다! ]는 업무일 확률 99.93%.

업무O.

[스토리 만들고, 스토리포인트 지정하고, 스프린트 시작하겠습니다! ]는 업무일 확률 99.98%.

업무O.

[배고파 쩝쩝]는 업무일 확률 0.06%.

업무X.

[식당좀 알아봐주세요]는 업무일 확률 83.89%.

업무O.

[오늘 회식있는데 괜찮은 식당 좀 알아봐주세요]는 업무일 확률 0.46%.

업무X.

[괜찮은 식당 좀 알아봐주세요]는 업무일 확률 16.02%.

업무X.

